## 🧠 Modélisation

Dans cette section, nous allons entamer la phase de modélisation, qui consiste à :

- Choisir un ou plusieurs modèles adaptés au type de problème (classification ou régression),
- Entraîner ces modèles sur l’ensemble d’apprentissage,
- Optimiser leurs performances à l’aide de la validation croisée et de la recherche d’hyperparamètres (GridSearchCV, RandomizedSearchCV).


In [ ]:
import pandas as pd
Path_Data='../Data/Processed/'
X_train = pd.read_csv(Path_Data+'X_train.csv')
X_val = pd.read_csv(Path_Data+'X_val.csv')
X_test = pd.read_csv(Path_Data+'X_test.csv')

y_train = pd.read_csv(Path_Data+'y_train.csv')
y_val = pd.read_csv(Path_Data+'y_val.csv')
y_test = pd.read_csv(Path_Data+'y_test.csv')


### Initialisation des Modèles 

Nous allons utiliser les modèles suivants pour nos tâches de classification et régression :

- **CatBoost**
- **LightGBM (LGBM)**
- **XGBoost**
- **Forêt Aléatoire (Random Forest)**
- **KNN (K-Nearest Neighbors)**
- Et d'autres modèles selon les besoins.


In [ ]:
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from LogisticRegression import LogisticRegression



models = {
    "Logistic Regression": LogisticRegression(),
    "XGBoost": XGBClassifier(),
    "CatBoost": CatBoostClassifier(verbose=0),
    "Random Forest": RandomForestClassifier(),
    "LightGBM": LGBMClassifier(),
    "KNN": KNeighborsClassifier()
}

##### On definie les Grids de Parametres Pour chaque model


In [ ]:
param_grids = {
    "Logistic Regression": {
        'alpha': [0.001, 0.01, 0.1],  
        'iterations': [500, 1000, 2000],  
        'use_l2': [True, False],  
        'lambda_': [0.01, 0.1, 1.0],  # L2 regularization strength
        'use_decay': [True, False],
        'decay': [0.001, 0.01, 0.1],
        'early_stopping': [True, False],
        'tol': [1e-4, 1e-5] 
    },
    "XGBoost": {
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.2],
        'n_estimators': [50, 100, 200]
    },
    "CatBoost": {
        'iterations': [500, 1000],
        'depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.2]
    },
    "Random Forest": {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, 30, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    "LightGBM": {
        'num_leaves': [31, 50, 100],
        'learning_rate': [0.01, 0.1, 0.2],
        'n_estimators': [50, 100, 200]
    },
    "KNN": {
        'n_neighbors': [3, 5, 7, 10],
        'weights': ['uniform', 'distance'],
        'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
    }
}

##  Détection de l’environnement d'exécution (CPU, GPU NVIDIA, ou puce Apple M1/M2..)

Ce script Python permet de **détecter automatiquement** l'environnement matériel sur lequel votre code est exécuté, afin d’adapter l'entraînement des modèles (par exemple : activer l’utilisation du GPU quand c’est possible).
Dans Notre Cas on a deux Puissante Machine l'une avec M1 Pro  et l'autre avec une Carte Graphique RTX 4070  

In [ ]:
import platform
import subprocess

def detect_environment():
    system = platform.system().lower()
    machine = platform.machine().lower()

    # Apple Silicons (M1/M2)
    if system == 'darwin' and 'arm' in machine:
        return "M1"

    # CUDA-compatible NVIDIA GPU
    try:
        result = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if result.returncode == 0:
            return "CUDA"
    except FileNotFoundError:
        pass

    return "CPU"

env = detect_environment()
print(f" Environnement détecté : {env}")


### 🔍 Optimisation des hyperparamètres avec GridSearchCV et accélération GPU

Pour garantir les meilleures performances de chaque modèle de classification, nous utilisons **GridSearchCV** pour effectuer un réglage fin des hyperparamètres. Voici les étapes :

1. **Exclusion des modèles personnalisés** :
   - Le modèle de régression logistique implémenté manuellement est exclu car il ne prend pas en charge `GridSearchCV` directement.

2. **Utilisation complète du CPU et du GPU** :
   - `n_jobs = -1` permet d’utiliser tous les cœurs du processeur pour les calculs parallèles.
   - Pour les modèles compatibles avec le GPU, nous activons explicitement l'accélération :
     - **XGBoost** : `tree_method='gpu_hist'`
     - **LightGBM** : `device='gpu'`
     - **CatBoost** : `task_type='GPU'`, `devices='0'`

3. **Affichage détaillé de l'entraînement** :
   - `verbose=2` affiche les étapes détaillées de l'entraînement, ce qui permet de suivre la progression en temps réel.

4. **Validation croisée** :
   - Une validation croisée à 3 plis (`cv=3`) est utilisée pour éviter le surapprentissage et améliorer la robustesse de la sélection des modèles.

5. **Évaluation** :
   - Après l'entraînement, nous extrayons les meilleurs hyperparamètres et évaluons le modèle sur l'ensemble de validation à l’aide de la précision (`accuracy`).




In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

for model_name, model in models.items():
    print(f"\n Démarrage de la GridSearch pour : {model_name}")

    if model_name == "Logistic Regression":
        print("  GridSearch ignorée pour la régression logistique personnalisée.")
        continue

    # Appliquer les bons paramètres selon l’environnement
    if env == "CUDA":
        if model_name == "XGBoost":
            model.set_params(tree_method='gpu_hist', predictor='gpu_predictor')
        elif model_name == "LightGBM":
            model.set_params(device='gpu')
        elif model_name == "CatBoost":
            model.set_params(task_type='GPU', devices='0', verbose=0)

    elif env == "M1":
        if model_name == "LightGBM":
            model.set_params(device_type='gpu')  # Metal backend
        elif model_name == "CatBoost":
            model.set_params(task_type='CPU', verbose=0)

    else:  # CPU fallback
        if model_name == "XGBoost":
            model.set_params(tree_method='hist', predictor='cpu_predictor')
        elif model_name == "LightGBM":
            model.set_params(device_type='cpu')
        elif model_name == "CatBoost":
            model.set_params(task_type='CPU', verbose=0)

    # Lancer GridSearchCV
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grids[model_name],
        cv=3,
        n_jobs=-1,
        verbose=2
    )
    
    grid_search.fit(X_train, y_train)

    print(f" Meilleurs hyperparamètres pour {model_name} : {grid_search.best_params_}")
    
    best_model = grid_search.best_estimator_
    y_pred_val = best_model.predict(X_val)
    val_acc = accuracy_score(y_val, y_pred_val)
    print(f" {model_name} - Accuracy Validation : {val_acc:.4f}")
